In [1]:
# Basics
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# TensorFlow/Keras (Keras layers below)
from tensorflow.keras.utils import set_random_seed, to_categorical
from tensorflow import keras
from keras.models import load_model
# set_random_seed(1234) # set the seeds for Python, NumPy, and TensorFlow in order to reproduce the results

# Custom
import sys
sys.path.insert(0, '../src/utils')
from trace_handler import TraceHandler
sys.path.insert(0, '../src/modeling')
from modeling_helpers import create_callbacks
from genetic_tuner import GeneticTuner


# Suppress TensorFlow messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # 1 for INFO, 2 for INFO & WARNINGs, 3 for INFO & WARNINGs & ERRORs

In [2]:
path = '/prj/side_channel/PinataTraces/CURR/D1-K1_50k_500MHz + Resampled at 168MHz.trs'

EM = True ##################
if EM: ##################
    path = '/prj/side_channel/PinataTraces/EM/EM02_D2-K1_50kTraces_500MHz + Resampled.trs' ##################

train_th = TraceHandler(path)    
    
BYTE_IDX = 0
N_CLASSES = 256
VAL_PERC = 0.1

x_train_tot = train_th.get_traces()
y_train_tot = train_th.get_specific_labels(BYTE_IDX)
y_train_tot = to_categorical(y_train_tot, N_CLASSES)

x_train, x_val, y_train, y_val = train_th.generate_train_val(BYTE_IDX, val_perc=VAL_PERC)
y_train = to_categorical(y_train, N_CLASSES)
y_val = to_categorical(y_val, N_CLASSES)

if EM: ##################
    x_test = x_val ##################
    y_test = y_val ##################
    
    x_val = x_train[-5000:] ##################
    y_val = y_train[-5000:] ##################

Labeling traces: 100%|██████████| 50000/50000 [00:02<00:00, 24063.58it/s]


In [3]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

HP_CHOICES = {#'kernel_initializer': ['random_normal', 'random_uniform', 
              #                       'truncated_normal', 
              #                       'zeros', 'ones', 
              #                       'glorot_normal', 'glorot_uniform',
              #                       'he_normal', 'he_uniform',
              #                       'identity', 'orthogonal', 'constant', 'variance_scaling'],
              'kernel_initializer': ['random_normal', 'random_uniform', 'truncated_normal', 'he_normal', 'he_uniform'],  
              'activation':         ['relu', 'tanh'],
              'hidden_layers':      [1, 2, 3, 4, 5],
              'hidden_neurons':     [100, 200, 300, 400, 500],
              'dropout_rate':       [0.0, 0.2, 0.4],
              'optimizer':          [SGD, Adam, RMSprop],
              'learning_rate':      [1e-3, 1e-4, 1e-5],
              'batch_size':         [50, 100, 200, 500]}    

In [ ]:
GENERATIONS = 2

gt = GeneticTuner(hp_choices=HP_CHOICES, pop_size=10, selection_perc=0.3)
population = gt.populate()

for i in range(GENERATIONS):
    print()
    print(f'-------------------- Generation {i+1}/{GENERATIONS} --------------------')
    
    evaluation = gt.evaluate(population, x_train, y_train, x_val, y_val) # list of tuple Network-val_acc
    
    parents = gt.select(evaluation) # list of Network objects
    
    if i != GENERATIONS - 1:
        population = gt.evolve(parents) # list of Network objects
    
    print(f'- Generation Results: {[ev[1] for ev in evaluation]}')


-------------------- Generation 1/2 --------------------
Training individual 1/10...
Training individual 2/10...
Training individual 3/10...
Training individual 4/10...
Training individual 5/10...
Training individual 6/10...
Training individual 7/10...
Training individual 8/10...
Training individual 9/10...
Training individual 10/10...
- Generation Results: [0.9865999817848206, 0.6747999787330627, 0.4050000011920929, 0.20479999482631683, 0.12240000069141388, 0.006599999964237213, 0.006200000178068876, 0.00559999980032444, 0.00559999980032444, 0.003800000064074993]

-------------------- Generation 2/2 --------------------
Training individual 1/10...
Training individual 2/10...
Training individual 3/10...
Training individual 4/10...
Training individual 5/10...
Training individual 6/10...


In [ ]:
# from datetime import datetime

best_individual, best_val_acc = evaluation[0]

# now = datetime.now()
# BEST_MODEL_PATH = '../saved_models/' + now.strftime('%m%d%Y_%H%M')
# best_individual.save_model(BEST_MODEL_PATH)

# print(f'Best val_acc: {best_val_acc}')

In [ ]:
if not EM:
    test_th = TraceHandler('/prj/side_channel/PinataTraces/CURR/D1-K2_50k_500MHz + Resampled at 168MHz.trs')

    x_test, y_test = test_th.generate_test(BYTE_IDX) 
    y_test = to_categorical(y_test, N_CLASSES)

    test_plaintexts = test_th.get_plaintexts()
    true_key_byte = test_th.get_key()[BYTE_IDX]
    
# test_th = TraceHandler('/prj/side_channel/PinataTraces/CURR/D1-K2_50k_500MHz + Resampled at 168MHz.trs')

# x_test, y_test = test_th.generate_test(BYTE_IDX) 
# y_test = to_categorical(y_test, N_CLASSES)

# test_plaintexts = test_th.get_plaintexts()
# true_key_byte = test_th.get_key()[BYTE_IDX]

In [ ]:
from tensorflow import keras
# from keras.models import load_model

# print('Loading the best model...')
# best_model = load_model(BEST_MODEL_PATH)

best_individual.final_train(x_train_tot, y_train_tot)

In [ ]:
if EM:
    test_plaintexts = train_th.get_plaintexts()[-5000:]
    true_key_byte = train_th.get_key()[BYTE_IDX]

best_individual.OLD_guessing_entropy(x_test, test_plaintexts, true_key_byte, BYTE_IDX)

In [ ]:
for hp in HP_CHOICES:
    print(f'{hp}: {best_individual.get_hp(hp)}')